In [1]:
from magent2.environments import battle_v4, adversarial_pursuit_v4, tiger_deer_v4
from pettingzoo.utils import random_demo
import torch
import time
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from dqn_basic import DQN_Basic
from collections import namedtuple, deque
import numpy as np
# import matplotlib.pyplot as plt
from Animal import Animal
from pathlib import Path
from collections import deque
import datetime, os
from metric_logger import MetricLogger
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# frame_size = (128, 128)
max_cycles = 300
total_episodes = 30
map_size = 40

# env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', tiger_step_recover=-0.1, deer_attacked=-0.1, max_cycles=max_cycles, extra_features=False)
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='rgb_array', max_cycles=max_cycles, extra_features=False)

print(env.action_spaces['deer_0'].n)
print(env.observation_spaces['deer_0'].shape)
deer_observation_shape = env.observation_spaces['deer_0'].shape
deer_observation_shape = tuple(np.roll(deer_observation_shape,1))
# deer_action_space = env.action_spaces['deer_0'].n
deer_action_space = 3

tiger_observation_shape = env.observation_spaces['tiger_0'].shape
tiger_observation_shape = tuple(np.roll(tiger_observation_shape,1))
# tiger_action_space = env.action_spaces['tiger_0'].n
tiger_action_space = 6

print(deer_observation_shape, tiger_observation_shape)
assert deer_observation_shape != None
assert tiger_observation_shape != None
# 'rgb_array' or 'human'
# random_demo(env, render=False , episodes=1)
# 'rgb_array'
# pygame.quit()

5
(13, 13, 5)
(5, 13, 13) (5, 13, 13)


c:\Users\Scott\a3\lib\site-packages\pettingzoo\utils\wrappers\base.py:64: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(
c:\Users\Scott\a3\lib\site-packages\pettingzoo\utils\wrappers\base.py:50: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(


In [3]:
# def deer_action_mapper(new_unmapped_action, prev_actual_action):
#     # 0 - forward, 1-left, 2-right
#     # 0 - up
#     # 1 - left
#     # 2 - stay still/  nothing
#     # 3 - right
#     # 4 - down
#     if new_unmapped_action == 0:
#         return prev_actual_action
#     if new_unmapped_action == 1:
#         if prev_actual_action == 0:
#             return 1
#         if prev_actual_action == 1:
#             return 4
#         if prev_actual_action == 3:
#             return 0
#         if prev_actual_action == 4:
#             return 3
#         return prev_actual_action
#     if new_unmapped_action == 2:
#         if prev_actual_action == 0:
#             return 3
#         if prev_actual_action == 1:
#             return 0
#         if prev_actual_action == 3:
#             return 4
#         if prev_actual_action == 4:
#             return 1
#     return 0
    

# def tiger_action_mapper(new_action, prev_action):
#     # new_action possibilities:
#     # 0 - forward, 1-left, 2-right
#     # 3 - attack forward, 4 - attack left, 5 - attack right
#     # prev_action possibilities:
#     # 0 - up
#     # 1 - left
#     # 2 - stay still/  nothing
#     # 3 - right
#     # 4 - down
#     # 5 - attack up
#     # 6 - attack left
#     # 7 - attack right
#     # 8 - attack down
#     direction = 0
#     if prev_action == 0 or prev_action == 5:
#         direction = 0
#     elif prev_action == 1 or prev_action == 6:
#         direction = 1
#     elif prev_action == 3 or prev_action == 7:
#         direction = 3
#     else:
#         direction = 4

#     if new_action == 0:
#         return direction
#     if new_action == 1:
#         if direction == 0:
#             return 1
#         if direction == 1:
#             return 4
#         if direction == 3:
#             return 0
#         if direction == 4:
#             return 3
#     if new_action == 2:
#         if direction == 0:
#             return 3
#         if direction == 1:
#             return 0
#         if direction == 3:
#             return 4
#         if direction == 4:
#             return 1
#     if new_action == 3:
#         if direction > 2:
#             return direction+4
#         return direction+5
#     if new_action == 4:
#         if direction == 0:
#             return 6
#         if direction == 1:
#             return 8
#         if direction == 3:
#             return 5
#         if direction == 4:
#             return 7
#     if new_action == 5:
#         if direction == 0:
#             return 7
#         if direction == 1:
#             return 5
#         if direction == 3:
#             return 8
#         if direction == 4:
#             return 6
#     return 0

deer_action_mapping = {
    0: {0: 0, 1: 1, 3: 3, 4: 4},
    1: {0: 1, 1: 4, 3: 0, 4: 3},
    2: {0: 3, 1: 0, 3: 4, 4: 1},
}
def deer_action_mapper(new_unmapped_action, prev_actual_action, size_of_actions=5):
    global deer_action_mapping
    return deer_action_mapping.get(new_unmapped_action, {}).get(prev_actual_action, np.random.choice(np.delete(np.arange(0, size_of_actions), 2)))

tiger_action_mapping = {
    0: {0: 0, 1: 1, 3: 3, 4: 4},
    1: {0: 1, 1: 4, 3: 0, 4: 3},
    2: {0: 3, 1: 0, 3: 4, 4: 1},
    3: {0: 5, 1: 6, 3: 7, 4: 8},
    4: {0: 6, 1: 8, 3: 5, 4: 7},
    5: {0: 7, 1: 5, 3: 8, 4: 6},
}

tiger_direction_mapping = {
    0: 0,
    1: 1,
    3: 3,
    4: 4,
    5: 0,
    6: 1,
    7: 3,
    8: 4,
}

def tiger_action_mapper(new_action, prev_action, size_of_actions=5): # slower
    global tiger_action_mapping, tiger_direction_mapping
    # tiger_action_mapping = {
    #     0: {0: 0, 1: 1, 3: 3, 4: 4},
    #     1: {0: 1, 1: 4, 3: 0, 4: 3},
    #     2: {0: 3, 1: 0, 3: 4, 4: 1},
    #     3: {0: 5, 1: 6, 3: 7, 4: 8},
    #     4: {0: 6, 1: 8, 3: 5, 4: 7},
    #     5: {0: 7, 1: 5, 3: 8, 4: 6},
    # }

    # tiger_direction_mapping = {
    #     0: 0,
    #     1: 1,
    #     3: 3,
    #     4: 4,
    #     5: 0,
    #     6: 1,
    #     7: 3,
    #     8: 4,
    # }

    direction = tiger_direction_mapping.get(prev_action, 0)
    return tiger_action_mapping.get(new_action, {}).get(direction, 0)
    

In [4]:
def get_agent_type_from_agent_name(agent):
    if 'tiger' in agent:
        return 'tiger'
    return 'deer'

In [5]:
def generate_first_action(agentType):
    if 'tiger' in agentType:
        # action = np.random.randint(0,tiger_action_space,(1,))[0]
        action = np.random.choice(np.arange(0, 6))
        actual_action = tiger_action_mapper(action, np.random.choice(np.delete(np.arange(0, 9), 2)))
    else:
        # action = np.random.randint(0,deer_action_space,(1,))[0]
        action = np.random.choice(np.arange(0, 3))
        actual_action = deer_action_mapper(action, np.random.choice(np.delete(np.arange(0, 5), 2)))
    
    return action, actual_action

### Trained Model

In [6]:
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()

deer_save_dir = Path("deer_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
deer_save_dir.mkdir(parents=True)

tiger_save_dir = Path("tiger_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
tiger_save_dir.mkdir(parents=True)

deer = Animal(state_dim=deer_observation_shape, action_dim=deer_action_space, save_dir=deer_save_dir)
tiger = Animal(state_dim=tiger_observation_shape, action_dim=tiger_action_space, save_dir=tiger_save_dir)

deer_logger = MetricLogger(deer_save_dir)
tiger_logger = MetricLogger(tiger_save_dir)

activeDeer = True
activeTiger = True
switching = False


episodes_per_switch = 20

for e in range(total_episodes):
    print("Episode: ", e)
    env.reset(seed=None)
    agents_prev_data = {}
    # Play the game!
    for agent in env.agent_iter():
        
        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)

        # instantiate previous data
        if agent not in agents_prev_data.keys():
            action, actual_action = generate_first_action(agentType)
            agents_prev_data[agent] = [observation, action, done, actual_action, reward,  info]
            env.step(actual_action)
            continue
        
        prev_observation, prev_action, prev_done, prev_actual_action = agents_prev_data[agent][0:4]
        
        if not prev_done:
            if 'tiger' in agentType:
                if activeTiger:
                    tiger.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                    #learn
                    q, loss = tiger.learn()
                    # logging
                    tiger_logger.log_step(reward, loss, q)
                action = tiger.act(observation)
                actual_action = tiger_action_mapper(action, prev_actual_action)
            else:
                if activeDeer:
                    deer.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                    #learn
                    q, loss = deer.learn()
                    # logging
                    deer_logger.log_step(reward, loss, q)
                action = deer.act(observation)
                actual_action = deer_action_mapper(action, prev_actual_action)
        
        if done:
            action = None
            actual_action = None
        
        agents_prev_data[agent] = [observation, action, done, actual_action, reward,  info]
        env.step(actual_action)
        

    deer_logger.log_episode()
    tiger_logger.log_episode()

    if e % 5 == 0:
        deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
        tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)
    if switching and e % episodes_per_switch == 0:
        activeDeer = not activeDeer
        activeTiger = not activeTiger
deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)

c:\Users\Scott\a3\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Using CUDA: True

Episode:  0
Animal saved to deer_checkpoints\2023-04-24T12-57-46\animal_net_0.chkpt at step 0
Animal saved to tiger_checkpoints\2023-04-24T12-57-46\animal_net_0.chkpt at step 0
Episode 0 - Step 8870 - Epsilon 0.9977849565598917 - Mean Reward -8.0 - Mean Length 8870.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 1.776 - Time 2023-04-24T12:57:48
Episode 0 - Step 1140 - Epsilon 0.9997150405729868 - Mean Reward 18.0 - Mean Length 1140.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 1.998 - Time 2023-04-24T12:57:48
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode 5 - Step 58466 - Epsilon 0.9854898006799002 - Mean Reward -12.333 - Mean Length 9744.333 - Mean Loss 0.001 - Mean Q Value -0.001 - Time Delta 85.671 - Time 2023-04-24T12:59:13
Episode 5 - Step 8282 - Epsilon 0.9979316417479602 - Mean Reward 23.667 - Mean Length 1380.333 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 85.651 - Time 2023-04-24T12:59:14
Episode:  6
Episode:  7
Episode:  8
Epis

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

### Visualizer of Trained Model

In [8]:
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', max_cycles=max_cycles, extra_features=False)
example_episodes = 10
for e in range(example_episodes):
    print("Episode: ", e)
    env.reset(seed=None)
    # Play the game!
    agents_prev_data = {}
    init_agent = None
    for agent in env.agent_iter():
        if init_agent is None:
            init_agent = agent
        else:
            if init_agent == agent:
                time.sleep(0.01)

        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)
        # instantiate previous data
        if agent not in agents_prev_data.keys():
            action, actual_action = generate_first_action(agentType)
            agents_prev_data[agent] = [observation, action, done, action, actual_action, info]
            env.step(actual_action)
            continue

        prev_actual_action = agents_prev_data[agent][3]
        if not done:
            if 'tiger' in agentType:
                # tiger.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                action = tiger.act(observation)
                actual_action = tiger_action_mapper(action, prev_actual_action)
            else:
                # deer.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                # action =
                action = deer.act(observation)
                actual_action = deer_action_mapper(action, prev_actual_action)
        else:
            action = None
            actual_action = None
        agents_prev_data[agent] = [observation, action, done, action, actual_action, info]
        env.step(actual_action)
    

Episode:  0
Episode:  1


SystemExit: 

c:\Users\Scott\a3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
